<a href="https://colab.research.google.com/github/mtermor/NTIC_DeepLearning/blob/main/NLP/02_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/eduardofc/data/main/amazon_sports.csv")
df['review_body'] = df['review_body'].str.replace('[^a-zA-ZñÑáéíóú .,:;]', '', regex=True)
df['review_body'] = df['review_body'].str.lower()
df.head()

,stars,review_body,review_title,product_category
0,1,nunca llego el pedido y el vendedor pasa de todo no contestan,No llego nunca,sports
1,1,"no sé como es, porque debería haber llegado ayer día de marzo, y hoy por la noche sigo esperando que llegue el frontal. y me habéis mandado el formulario de opinión. pues de momento mala no ha cumplido con la fecha de entrega.",Todavía no ha llegado,sports
2,1,"guantes cómodos, no lo niego, pero de mala calidad. yo creo que en caso de caída no valdrian para mucho, dos meses de uso y se están rajando.",Guantes de baja calidad,sports
3,1,hasta hoy no he visto el producto. el pedido hace ya casi mes. y notifico que he usado prime para está compra.,Muy Mala experiencia,sports
4,1,"no puedo valorarla porque, después de casi una semana, aún no he recibido mi pedido. pienso que amazon tendría que valorar las compañías de transporte con que trabaja, porque es indignante que pague mi cuota prime y nunca reciba mi pedido el día que toca",Paquete perdido?,sports


In [3]:
df.groupby('stars').size()

stars
1    2438
2    2551
3    2828
4    2860
5    2512
dtype: int64

In [4]:
""" Clasificamos en positivo (stars 4-5) y negativo (stars 1-2) """
df = df[df.stars != 3]
df['bad_product'] = (df.stars < 3).astype(int)
df.groupby('bad_product').size()

bad_product
0    5372
1    4989
dtype: int64

# Modelling for Classification

## Inputs (preprocessing)

In [5]:
X = df.review_body.values
y = df.bad_product

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer

vocab_size = 10000
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X)
# tokenizer.index_word
X_tokenized = tokenizer.texts_to_sequences(X)
# X_tokenized

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 50
X_padded = pad_sequences(X_tokenized, maxlen=max_len, truncating='post')
# X_padded

## Model (NN)

In [8]:
import tensorflow.keras as keras

keras.utils.set_random_seed(812)

In [9]:
from keras import Sequential
from keras.layers import Flatten, Dense, Embedding

embedding_dim = 2

model = keras.Sequential()

# Capas de embeddings
model.add(Embedding(input_length= max_len, input_dim=vocab_size, output_dim=embedding_dim))
model.add(Flatten())
# Capas de clasificación
# model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [10]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 2)             20000     
                                                                 
 flatten (Flatten)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 20101 (78.52 KB)
Trainable params: 20101 (78.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
n_epochs = 20

model.fit(X_padded, y, epochs= n_epochs)

Epoch 1/20
324/324 [==============================] - 25s 67ms/step - loss: 0.6621 - accuracy: 0.6195
Epoch 2/20
324/324 [==============================] - 5s 15ms/step - loss: 0.5053 - accuracy: 0.7949
Epoch 3/20
324/324 [==============================] - 3s 9ms/step - loss: 0.3877 - accuracy: 0.8505
Epoch 4/20
324/324 [==============================] - 2s 7ms/step - loss: 0.3293 - accuracy: 0.8751
Epoch 5/20
324/324 [==============================] - 3s 10ms/step - loss: 0.2912 - accuracy: 0.8906
Epoch 6/20
324/324 [==============================] - 2s 6ms/step - loss: 0.2626 - accuracy: 0.9024
Epoch 7/20
324/324 [==============================] - 2s 7ms/step - loss: 0.2387 - accuracy: 0.9147
Epoch 8/20
324/324 [==============================] - 2s 7ms/step - loss: 0.2188 - accuracy: 0.9239
Epoch 9/20
324/324 [==============================] - 1s 4ms/step - loss: 0.2021 - accuracy: 0.9296
Epoch 10/20
324/324 [==============================] - 2s 5ms/step - loss: 0.1874 - accuracy: 0.